# Noiseprint Demo
Este notebook apresenta o fluxo do protótipo Noiseprint: da preparação do ambiente à análise qualitativa dos mapas gerados (fallback vs. pesos reais, quando disponíveis).

In [2]:
from pathlib import Path
import logging
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

ROOT = Path.cwd().resolve().parent
SRC_PATH = ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

DATA_INPUT = ROOT / "data" / "input"
DATA_OUTPUT = ROOT / "data" / "output"
DATA_INPUT.mkdir(parents=True, exist_ok=True)
DATA_OUTPUT.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

ModuleNotFoundError: No module named 'cv2'

In [3]:
from noiseprint import compute_noiseprint, load_model
from noiseprint.filters import normalize_map

image_paths = [
    p for p in sorted(DATA_INPUT.glob("*"))
    if p.suffix.lower() in {".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp"}
]

if not image_paths:
    raise FileNotFoundError(
        "Nenhuma imagem encontrada em data/input. Adicione 2–3 arquivos antes de seguir."
    )

selected_paths = image_paths[:3]
print("Imagens selecionadas:")
for path in selected_paths:
    print(f"- {path.name}")

def load_bgr(path: Path) -> np.ndarray:
    img = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img is None:
        raise IOError(f"Falha ao carregar {path}")
    return img

IMAGES = [(path, load_bgr(path)) for path in selected_paths]

ModuleNotFoundError: No module named 'noiseprint'

In [4]:
from noiseprint.visualize import save_heatmap, save_overlay
from matplotlib import cm

model = load_model()
if model is not None:
    model = model.to(device)
    print("Modelo carregado e movido para", device)
else:
    print("Sem pesos: usando residual fallback.")

results = []
for path, img_bgr in IMAGES:
    fallback_map = compute_noiseprint(img_bgr, model=None)
    model_map = compute_noiseprint(img_bgr, model=model) if model is not None else None

    def make_overlay(image_bgr: np.ndarray, residual: np.ndarray, alpha: float = 0.6):
        heatmap = cm.get_cmap("jet")(normalize_map(residual))[..., :3]
        heatmap_rgb = (heatmap * 255).astype(np.uint8)
        heatmap_bgr = cv2.cvtColor(heatmap_rgb, cv2.COLOR_RGB2BGR)
        if heatmap_bgr.shape[:2] != image_bgr.shape[:2]:
            heatmap_bgr = cv2.resize(heatmap_bgr, (image_bgr.shape[1], image_bgr.shape[0]))
        overlay = cv2.addWeighted(
            image_bgr.astype(np.float32), 1.0 - alpha,
            heatmap_bgr.astype(np.float32), alpha, 0
        )
        return overlay.astype(np.uint8), heatmap_rgb

    fallback_overlay_bgr, fallback_heatmap_rgb = make_overlay(img_bgr, fallback_map)
    model_overlay_bgr = None
    model_heatmap_rgb = None
    if model_map is not None:
        model_overlay_bgr, model_heatmap_rgb = make_overlay(img_bgr, model_map)

    results.append(
        {
            "path": path,
            "image_bgr": img_bgr,
            "fallback_map": fallback_map,
            "fallback_overlay_bgr": fallback_overlay_bgr,
            "fallback_heatmap_rgb": fallback_heatmap_rgb,
            "model_map": model_map,
            "model_overlay_bgr": model_overlay_bgr,
            "model_heatmap_rgb": model_heatmap_rgb,
        }
    )

for item in results:
    img_rgb = cv2.cvtColor(item["image_bgr"], cv2.COLOR_BGR2RGB)
    fallback_overlay_rgb = cv2.cvtColor(item["fallback_overlay_bgr"], cv2.COLOR_BGR2RGB)

    columns = 3 if item["model_map"] is not None else 2
    fig, axes = plt.subplots(1, columns, figsize=(4 * columns, 4))
    axes = np.atleast_1d(axes)

    axes[0].imshow(img_rgb)
    axes[0].set_title(f"Original: {item['path'].name}")
    axes[0].axis('off')

    axes[1].imshow(item["fallback_heatmap_rgb"])
    axes[1].set_title("Heatmap fallback")
    axes[1].axis('off')

    if item["model_map"] is not None and columns == 3:
        axes[2].imshow(item["model_heatmap_rgb"])
        axes[2].set_title("Heatmap modelo")
        axes[2].axis('off')

    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(4 * columns, 4))
    plt.subplot(1, columns, 1)
    plt.imshow(img_rgb)
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, columns, 2)
    plt.imshow(fallback_overlay_rgb)
    plt.title("Overlay fallback")
    plt.axis('off')

    if item["model_overlay_bgr"] is not None and columns == 3:
        overlay_rgb = cv2.cvtColor(item["model_overlay_bgr"], cv2.COLOR_BGR2RGB)
        plt.subplot(1, columns, 3)
        plt.imshow(overlay_rgb)
        plt.title("Overlay modelo")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

ModuleNotFoundError: No module named 'noiseprint'

### Observações rápidas
- Regiões com alto contraste (bordas, texturas) costumam gerar respostas fortes no fallback, pois o residual é basicamente um filtro passa-alta.
- Quando pesos reais são usados, espera-se redução de falso-positivos em detalhes finos e maior consistência entre imagens da mesma câmera.
- A abordagem manual é útil para prototipagem, mas não captura o fingerprint específico de cada sensor.

In [ ]:
for item in results:
    base = item["path"].stem
    heatmap_path = DATA_OUTPUT / f"{base}_heatmap_demo.png"
    overlay_path = DATA_OUTPUT / f"{base}_overlay_demo.png"
    save_heatmap(item["fallback_map"], str(heatmap_path))
    save_overlay(item["image_bgr"], item["fallback_map"], str(overlay_path))
    if item["model_map"] is not None:
        heatmap_model_path = DATA_OUTPUT / f"{base}_heatmap_model.png"
        overlay_model_path = DATA_OUTPUT / f"{base}_overlay_model.png"
        save_heatmap(item["model_map"], str(heatmap_model_path))
        save_overlay(item["image_bgr"], item["model_map"], str(overlay_model_path))

print("Resultados exportados para", DATA_OUTPUT)